In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
# Read into csv file
schools_csv_file = "./Resources/dv279-schoollocations2019.csv"
schools_df = pd.read_csv(schools_csv_file)
schools_df.head()

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [3]:
# Drop unwanted columns
to_drop = ["School_Status",
          "Address_Line_1",
          "Address_Line_2",
          "Address_Town",
          "Address_State",
          "Postal_Address_Line_2",
          "Full_Phone_No",
          "Address_Postcode",
          "Postal_State",
          "LGA_ID"]

schools_df.drop(columns=to_drop, inplace=True)

In [4]:
schools_df.head()

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,Postal_Address_Line_1,Postal_Town,Postal_Postcode,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,21 Thomson Street,ALBERTON,3971,Wellington (S),146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,Frank Street,ALLANSFORD,3277,Warrnambool (C),142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,P O Box 12,AVOCA,3467,Pyrenees (S),143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,40 Anderson Street,AVENEL,3664,Strathbogie (S),145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,5-11 Forbes Street,WARRANDYTE,3113,Manningham (C),145.213980,-37.742675


In [5]:
# Split LGA Name
schools_df["LGA_Name"] = schools_df.LGA_Name.apply(lambda x: pd.Series(str(x).split(" "))) 

In [6]:
# Capitalise and lower postal town names
schools_df["Postal_Town"] = schools_df.Postal_Town.apply(lambda x: pd.Series(str(x).lower().title()))

In [7]:
schools_df.head()

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,Postal_Address_Line_1,Postal_Town,Postal_Postcode,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,21 Thomson Street,Alberton,3971,Wellington,146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,Frank Street,Allansford,3277,Warrnambool,142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,P O Box 12,Avoca,3467,Pyrenees,143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,40 Anderson Street,Avenel,3664,Strathbogie,145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,5-11 Forbes Street,Warrandyte,3113,Manningham,145.213980,-37.742675


In [8]:
# Rename postcode column
schools_df = schools_df.rename(columns={"Postal_Postcode": "Postcode",
                                       "Postal_Town": "Suburb",
                                       "X": "Lat",
                                       "Y": "Lng"})
schools_df

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,Postal_Address_Line_1,Suburb,Postcode,LGA_Name,Lat,Lng
0,Government,1,1,Alberton Primary School,Primary,21 Thomson Street,Alberton,3971,Wellington,146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,Frank Street,Allansford,3277,Warrnambool,142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,P O Box 12,Avoca,3467,Pyrenees,143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,40 Anderson Street,Avenel,3664,Strathbogie,145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,5-11 Forbes Street,Warrandyte,3113,Manningham,145.213980,-37.742675
...,...,...,...,...,...,...,...,...,...,...,...
2249,Catholic,2,2222,Our Lady Star of the Sea Catholic Primary School,Primary,6 Cowes-Rhyll Road,Cowes,3922,Bass,145.239805,-38.463698
2250,Independent,2,2223,Maxwell Creative School,Primary,6 Rainy Hill Road,Cockatoo,3781,Cardinia,145.518028,-37.923946
2251,Catholic,2,2224,St Anne's College,Pri/Sec,72 Wendouree Drive,Kialla,3631,Greater,145.400337,-36.436121
2252,Catholic,2,2225,Holy Trinity Catholic Primary School,Primary,145 Mitchells Lane,Sunbury,3429,Hume,144.705316,-37.584791


In [10]:
# Retrieve postcode and suburbs
suburb_lga_df = schools_df[["Suburb", "LGA_Name"]]
suburb_lga_df = suburb_lga_df.drop_duplicates()

# Export into a csv file
suburb_lga_df.to_csv("./output/suburb_lga_df.csv", index=False)

In [11]:
# Read into created csv
suburb_lga = pd.read_csv("./output/suburb_lga_df.csv")
suburb_lga

,Suburb,LGA_Name
0,Alberton,Wellington
1,Allansford,Warrnambool
2,Avoca,Pyrenees
3,Avenel,Strathbogie
4,Warrandyte,Manningham
...,...,...
962,Truganina South,Wyndham
963,Richmond,Melbourne
964,Koonwarra,Bass
965,Rosebud West,Mornington


In [12]:
cowes = suburb_lga.loc[suburb_lga["Suburb"] == "Cowes"]
cowes

,Suburb,LGA_Name
198,Cowes,Bass


In [ ]:
# Groupby suburbs
grouped_schools_df = schools_df.groupby(["Suburb"])

# Find number of schools in each suburb
schools_count = grouped_schools_df["School_Name"].count()
schools_count

# Create a new df with school count
schools_count_df = schools_count.reset_index()
schools_count_df

# Rename column
schools_count_df = schools_count_df.rename(columns={"Suburb": "suburb",
                                                    "School_Name": "School Count"})
schools_count_df

In [ ]:
# Import pre-covid housing data
pre_covid_sales_file = "./MT_resources/vic-sales-precovid.csv"
pre_covid_sales_df = pd.read_csv(pre_covid_sales_file)
pre_covid_sales_df

In [ ]:
# Merge suburb school counts into pre-covid housing data
merged_pre_covid = pd.merge(schools_count_df, pre_covid_sales_df, how="left", on="suburb")
merged_pre_covid

In [ ]:
# Check pre covid data for nulls
merged_pre_covid.count()

In [ ]:
# Drop null values
merged_pre_covid = merged_pre_covid.dropna()
merged_pre_covid

In [ ]:
# Rename columns for consistency
merged_pre_covid = merged_pre_covid.rename(columns={"Postcode": "postcode",
                                                    "School Count": "school_count"})
merged_pre_covid

In [ ]:
# Groupby on suburbs
grouped_suburbs_df = merged_pre_covid.groupby(["suburb"])

# Normalise housing prices in each suburb by finding the average purchase
grouped_suburbs_df = round(grouped_suburbs_df[["school_count", "price"]].mean(),2)

grouped_suburbs_df

In [ ]:
# Visualise
grouped_suburbs_df.plot(kind="scatter", x="school_count", y="price", grid=True, title="Number of Schools vs Average Housing Price in Victorian Suburbs")

In [ ]:
# Linear regression function
def create_linear_regression_plot(x_values, y_values, x_label, y_label, period, line_placement, ylim=None):
    (slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_values, y_values)
    
    # Get regression values
    regress_values = x_values * slope + intercept
    
    # Create line equation string
    line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept, 2))
    
    # Using pyplot to create scatter plot
    plt.scatter(x_values, y_values)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    if ylim != None:
        plt.ylim = (0, ylim)
    plt.title(f'{period} covid - {x_label} vs. {y_label}')
    plt.annotate(line_eq,line_placement,fontsize=15,color="red")
    
    # Print r-squared value
    print(f"The r-squared is: {rvalue**2}")
    
    # Print correlation between both factors
    correlation = st.pearsonr(x_values,y_values)
    print(f"The correlation between both factors is {round(correlation[0],2)}")
    
    # Save linear regression plot as png file
    #plt.savefig(f"./output/{period}_{x_label}_vs_{y_label}.png")
    
    return plt.plot(x_values,regress_values,"r-")

In [ ]:
x_values = grouped_suburbs_df["school_count"]
y_values = grouped_suburbs_df["price"]
create_linear_regression_plot(x_values, y_values, "Number of Schools", "Average Housing Price ($)", "Pre", (10,1.5), ylim=None)